Tristan MANIER - Léa GRASSIEN - Mathieu RAMEL - INFO 5 - 9/01/2024


# 1. Présentation des données

Notre dataset contient deux fichiers : 
- eCO2mix_RTE_En-cours-TR.csv :  https://www.rte-france.com/eco2mix
- donnees-synop-essentielles-omm.csv :  

Le fichier eCO2mix_RTE_En-cours-TR.csv contient les données de consommation et de production en électricité en France, les échanges d'éléctricité avec d'autres pays Européens sur une période allant de juin 2022 à début 2023.
Le fichier donnees-synop-essentielles-omm.csv contient les données météorologiques de la France sur les années 2022 à début 2024.

Nous avons choisi ces données parce que ce sont des données d'actualité. En effet avec l'augmentation des prix et le changement climatique, il est très intéressant de se pencher sur ce genre de données. Les données de consommation et de production d'électricité sont aussi très intéressantes en général. Il est possible d'obtenir des informations très utiles sur les habitudes de consommation et la façon de faire des éconnomies.

### Description plus précise des données
|         | nombre de variables | nombre de lignes |                 Granularité                 |                                      Informations de la table                                       |
|---------|:-------------------:|:----------------:|:-------------------------------------------:|:---------------------------------------------------------------------------------------------------:|
| Eco2mix |         40          |      56640       | 2022-06-01 / 2024-01-14  Précision de 15min | Consommation, découpage de la production (Gaz, Fioul, ...), échange d'électicité avec d'autres pays |
| Météo   |         59          |      341186      |   2022-01-01 / 2024-01-14 Précision de 3h   |                                    Pluie, vent, température, ...                                    |


## Questions

### Analyse
Nous allons chercher à répondre à ces questions :
- Quelles sont les sources de production d'énergie les plus corrélées ?
- Quelles est la proportion d'énergie renouvelable produite ?
- Quelles sont les tendances de méthode de production d'énergie au cours de l'année ?
- Quelles sont les périodes de l'année où la production d'énergie est la plus importante/faible ?
- Quelles sont les moments de la journée où la production d'énergie est la plus importante/faible ?
- La météo influe-t-elle sur la consomation d'énergie ?
- Quelles sont les sources de production d'énergie les plus corrélées avec la météo ?
- Classification des jousrs de l'année par rapport à la consomation d'énergie.

### Prédictions
- Quelle est la tendance de production d'énergie pour les jours/mois/années à venir ?
- Peut-on classifier les jours de la semaine par rapport à la consomation et la production d'énergie ?

# 2. Préparation des données

Modules

In [ ]:
import math
import pandas as pd

pd.set_option(
    "display.max_rows", 100
)  # Set the maximum number of rows displayed to 100 rows

## 2.1 Premier dataset sur la production d'énergie

Importation des données

In [ ]:
file_name = "eCO2mix_RTE_En-cours-TR.csv"

df = pd.read_csv(file_name, sep="\t", encoding="latin-1", index_col=False)

df.head()

Renommage des colonnes pour retirer les espaces et faciliter la selection ensuite

In [ ]:
df.columns = [c.replace(" ", "_") for c in df.columns]
df.head()

### 2.1.1 Correction des données

Affichage des données non-définies et des types des colonnes pour vérifier la qulaité de notre dataset.

Ici on remarque que certaines colonnes contiennent beaucoup de données non-définies et que d'autres ont des types object.

Il faut corriger cela.

In [ ]:
temp = {"column": [], "type": [], "nb_lines": [], "nb_ND": []}

nd = pd.DataFrame(temp)

for column in df.columns:
    new_row = pd.DataFrame(
        {
            "column": column,
            "type": df[column].dtype,
            "nb_lines": df[column].count(),
            "nb_ND": (df[column] == "ND").sum(),
        },
        index=[0],
    )
    nd = pd.concat([new_row, nd.loc[:]]).reset_index(drop=True)
nd = nd.sort_values(by="nb_ND", ascending=False)

nd["%ND"] = nd["nb_ND"] / nd["nb_lines"] * 100

nd

Commençons par Supprimer les lignes sans dates et heures.

In [ ]:
df.dropna(subset=["Date", "Heures"], inplace=True)

Suppression des lignes sans information de consommation et sans prédiction, celles qui n'apportent aucune information concrète. 

In [ ]:
df = df[df["Prévision_J-1"] != "ND"]
df = df[:-1]

Supression de colonnes décomposant la production que nous n'utiliseront pas :

- Hydrolique est décomposé en 3 catégories : 'Hydraulique - Fil de l?eau + éclusée', 'Hydraulique - Lacs', 'Hydraulique - STEP turbinage' \
- Bioénergies est décomposé en 3 catégories : 'Bioénergies - Déchets', 'Bioénergies - Biomasse', 'Bioénergies - Biogaz' \
- Eolien est décomposé en 2 catégories : 'Eolien terrestre', 'Eolien offshore' \
- Fioul est décomposé en 3 catégories : 'Fioul - TAC', 'Fioul - Cogén.', 'Fioul - Autres' \
- Gaz est décomposé en 3 catégories :  'Gaz - TAC', 'Gaz - Cogén.', 'Gaz - CCG', 'Gaz - Autres' 
- Les deux colonnes sur les batteries ne semble pas pertinente non plus.

In [ ]:
list_energy = [
    "Périmètre",
    "Date",
    "Heures",
    "Consommation",
    "Prévision_J-1",
    "Prévision_J",
    "Fioul",
    "Charbon",
    "Gaz",
    "Nucléaire",
    "Eolien",
    "Solaire",
    "Hydraulique",
    "Pompage",
    "Bioénergies",
    "Ech._physiques",
    "Taux_de_Co2",
    "Ech._comm._Angleterre",
    "Ech._comm._Espagne",
    "Ech._comm._Italie",
    "Ech._comm._Suisse",
    "Ech._comm._Allemagne-Belgique",
]

for col_name in df.columns:
    if col_name not in list_energy:
        df = df.drop(col_name, axis=1)

In [ ]:
print(f"{df.shape[0]} rows and {df.shape[1]} columns")

# Combine dtypre, count and nnunique
pd.concat(
    [df.dtypes, df.count(), df.nunique()], keys=["Types", "Count", "NUnique"], axis=1
)

Ici on observe que les types des colonnes restantes sont plus logiques, on obtient plus de float au lieu de type object sur des colonnes decrivant une consommation ou une production.

### 2.1.2 Ajout de colonnes

Ajout d'une colonne production totale qui est la somme de toutes les productions.

In [ ]:
columns_to_sum = [
    "Fioul",
    "Charbon",
    "Gaz",
    "Nucléaire",
    "Eolien",
    "Solaire",
    "Hydraulique",
    "Pompage",
    "Bioénergies",
]
df["Production"] = df[columns_to_sum].sum(axis=1)

### Statistiques descriptives des valeurs non-définies

In [ ]:
temp = {"column": [], "nb_lines": [], "nb_ND": []}

nd = pd.DataFrame(temp)

for column in df.columns:
    new_row = pd.DataFrame(
        {
            "column": column,
            "nb_lines": df[column].count(),
            "nb_ND": (df[column] == "ND").sum(),
        },
        index=[0],
    )
    nd = pd.concat([new_row, nd.loc[:]]).reset_index(drop=True)
nd = nd.sort_values(by="nb_ND", ascending=False)

nd["%ND"] = nd["nb_ND"] / nd["nb_lines"] * 100

nd

## 2.2 Deuxième dataset sur les données météo

Importation des données météo

In [ ]:
file_name = "weather.csv"

df_weather = pd.read_csv(file_name, sep=";", index_col=False, encoding="utf-8")

# df_weather.head()
print(f"{df_weather.shape[0]} rows and {df_weather.shape[1]} columns")

df_weather.head()

Selection des données d'une seule région. (Pourrais être remplacer par une moyenne des régions) 

In [ ]:
code_region = int("07149")  # Ile-de-France
df_weather = df_weather[df_weather["numer_sta"] == code_region]

# df_weather.head()
print(f"{df_weather.shape[0]} rows and {df_weather.shape[1]} columns")

Formattage des données météo.

Parmis toutes les données météo fournies, nous avons choisis de ne garder que les données les plus pertinantes pour nous. De plus, nous les avons renomer pour facilité leur utilisation et leur lisibilité.

In [ ]:
# multiplier les colonnes par 3 pour avoir 1 lignes par heures
weather_rename_map = {
    "date": "date",
    "pmer": "pression_mer",
    "ff": "vitesse_vent",
    "t": "température",
    "u": "humidité",
    "pres": "pression",
    "niv_bar": "niveau_barometrique",
    "tn12": "t_min_12h",
    "tn24": "t_min_24h",
    "tx12": "t_max_12h",
    "tx24": "t_max_24h",
    "tminsol": "t_min_sol_12h",
    "rr1": "précipitation_1h",
    "rr3": "précipitation_3h",
    "rr6": "précipitation_6h",
    "rr12": "précipitation_12h",
    "rr24": "précipitation_24h",
    "ssfrai": "hauteur_neige",
}

df_weather = df_weather.rename(columns=weather_rename_map)

# remove columns not in weather_rename_map
for col_name in df_weather.columns.values.tolist():
    if col_name not in weather_rename_map.values():
        df_weather = df_weather.drop(col_name, axis=1)

df_weather = df_weather.reset_index(drop=True)

# convert object to float
for col_name in df_weather.columns.values.tolist():
    df_weather[col_name] = pd.to_numeric(df_weather[col_name], errors="coerce")

# df_weather.head()

# df_weather.dtypes

Suppresion des lignes avec des valeurs manquantes pour les dates et heures.

In [ ]:
df = df.dropna(subset=["Date", "Heures"])

## 2.3 Jointure des deux datasets

Formatage des dates et heures des deux datasets.

In [ ]:
# Merge Date and Heures
df["date"] = df["Date"].astype(str) + df["Heures"].astype(str)

# Convert to datetime
df["date"] = df["date"].apply(lambda x: pd.to_datetime(str(x), format="%Y-%m-%d%H:%M"))

# Convert to datetime
df_weather["date"] = df_weather["date"].apply(
    lambda x: pd.to_datetime(str(x), format="%Y%m%d%H%M%S")
)

Fusion des deux datasets

In [ ]:
df_full = pd.merge(df, df_weather, how="inner", on="date")

# full_df.head()
print(f"df : {df.shape[0]} rows and {df.shape[1]} columns")
print(f"df_weather : {df_weather.shape[0]} rows and {df_weather.shape[1]} columns")
print(f"df_full : {df_full.shape[0]} rows and {df_full.shape[1]} columns")

print(f"Data between {df_full['date'].min()} and {df_full['date'].max()}")

In [ ]:
df_full.head()

# 3. Analyse des données

Selection d'une periode de temps

In [ ]:
from matplotlib import pyplot as plt

# select rows between date1 and date 2
date1 = pd.to_datetime("2022-06-01")
date2 = pd.to_datetime("2024-09-01")

df_selection = df_full[(df_full["date"] >= date1) & (df_full["date"] < date2)]

### 3.1 La période a-t-elle une influence sur la production et la consommation ?

Visualisation des données de consomations

In [ ]:
figure = plt.figure(figsize=(20, 10))
plt.plot(df_selection["date"], df_selection["Consommation"], label=column)
plt.legend(loc="upper right", numpoints=1)
plt.xlabel("Date")
plt.ylabel("Production (MW)")
plt.show()

Visualisation des données de production

In [ ]:
to_plot = [
    "Charbon",
    "Eolien",
    "Solaire",
    "Nucléaire",
    "Hydraulique",
    "Bioénergies",
    "Pompage",
]

figure = plt.figure(figsize=(20, 10))
for column in to_plot:
    plt.plot(df_selection["date"], df_selection[column], label=column)
    plt.legend(loc="upper right", numpoints=1)

plt.xlabel("Date")
plt.ylabel("Production (MW)")
plt.show()

In [ ]:
to_plot = ["Consommation", "Production"]

figure = plt.figure(figsize=(20, 10))
for column in to_plot:
    plt.plot(df_selection["date"], df_selection[column], label=column)
    plt.legend(loc="upper right", numpoints=1)

plt.xlabel("Date")
plt.ylabel("(MW)")
plt.show()

In [ ]:
bin_size = 6

figure = plt.figure(figsize=(20, 10))
for i in range(0, 24, bin_size):
    lab = ""
    to_plot = pd.DataFrame()
    for j in range(1, bin_size):
        lab += str(i + j) + " "
        to_plot = pd.concat(
            [
                to_plot,
                df_selection[pd.to_datetime(df_selection["Heures"]).dt.hour == i + j],
            ]
        )

    lab += " heures"

    plt.plot(to_plot["date"], to_plot["Consommation"], label=lab)
    plt.legend(loc="upper right", numpoints=1, title="Interval d'heures")
# plt.plot(df.groupby([ time_date , time_date.dt.hour 'heure']).Consommation.sum())
plt.xlabel("Date")
plt.ylabel("(MW)")
plt.show()

### 3.2 Quelles sont les sources de production d'énergie les plus corrélées ?

Selections des colonnes liées à la production et description de celles-ci

In [ ]:
df_energies = df_full[
    [
        "Charbon",
        "Gaz",
        "Fioul",
        "Nucléaire",
        "Eolien",
        "Solaire",
        "Hydraulique",
        "Bioénergies",
    ]
]

df_energies.describe()

Matrice de corrélation

In [ ]:
df_energies.corr()

#### Conclusion
On remarque que les sources les plus corrélées sont de loin le gaz et le charbon ainsi que le nucléaire et l'hydraulique.
Derrière on retrouve le gaz et le fioul qui ont une corrélation assez basse comparé aux deux premiers mais qui reste notable.

Le charbon et le gaz  et le fioul sont toutes les trois des sources non renouvelables ce qui pourrait expliquer leur corrélations. 
Pour ce qui est du nucléaire et de l'hydraulique cela devient plus intriguant.
Les centrales nucléaires nécessitent une grande quantité d'eau ce qui pourrait avoir un impact sur l'hydraulique.

En revanche ce que l'on peut conclure est qu'aucune énergies ne sont vraiment corrélées ensemble car la valeur la plus haute reste en dessous de 0.6. Cela fait sens car ces industries ont tendances à être relativement indépendantes.

### 3.3 Quelles est la proportion d'énergie renouvelable produite ?

Calcul de la production globale d'énergie sur la période du dataset

In [ ]:
name_col = [
    "Charbon",
    "Gaz",
    "Fioul",
    "Nucléaire",
    "Eolien",
    "Solaire",
    "Hydraulique",
    "Bioénergies",
]

df_energies = df_full[name_col]

df_energies = df_energies.dropna()

total_energies = sum(df_energies.sum(axis=1))
print(f"Production total : {total_energies} MW")

Calcul des proportions de chaque source d'énergie

In [ ]:
sum_dic = {}
pourcent_dic = {}
for col in name_col:
    sum_dic[col] = sum(df_energies[col])
    pourcent_dic[col] = sum_dic[col] / total_energies * 100

pourcent_df = pd.DataFrame.from_dict(pourcent_dic, orient="index", columns=["%"])
print(pourcent_df)

Aggrégation des sources d'énergie en 3 catégories : renouvelable, fossile et nucléaire

In [ ]:
renouvelable = ["Eolien", "Solaire", "Hydraulique", "Bioénergies"]
pourcent_renouvelable = sum([pourcent_dic[nrj12] for nrj12 in renouvelable])
print(f"Pourcentage d'énergie renouvelable : {pourcent_renouvelable:.2f} %")

fossil = ["Charbon", "Gaz", "Fioul"]
pourcent_fossil = sum([pourcent_dic[nrj12] for nrj12 in fossil])
print(f"Pourcentage d'énergie fossile : {pourcent_fossil:.2f} %")

nuclear = ["Nucléaire"]
pourcent_nuclear = sum([pourcent_dic[nrj12] for nrj12 in nuclear])
print(f"Pourcentage d'énergie nucléaire : {pourcent_nuclear:.2f} %")

#### Conclusion
Les énergies renouvelables représente plus d'un quart de la production total d'énergie.
Cependant ce nombre fait pale figure face au 65% que représente le nucléaire.
Il est important de remarqué que de nos jours, les énergies renouvelables sont plus de 4x supérieur à la production d'énergies fossile (non renouvelables).

Pour conclure, ce nombre reste quand même conséquent et reste logique pour un pays comme la france qui s'appuis beaucoup sur le nucléaire.


### 3.4 Quelles sont les tendances de méthode de production d'énergie au cours de l'année ?

Séparation des données par mois de l'année.

In [ ]:
date1 = pd.to_datetime("2023-01-01")
date2 = pd.to_datetime("2024-01-01")
df_selection = df_full[(df_full["date"] >= date1) & (df_full["date"] < date2)]

name_col = [
    "date",
    "Charbon",
    "Gaz",
    "Fioul",
    "Nucléaire",
    "Eolien",
    "Solaire",
    "Hydraulique",
    "Bioénergies",
]

df_energies = df_full[name_col]

df_energies = df_energies.dropna()


mois = [
    "Janvier",
    "Février",
    "Mars",
    "Avril",
    "Mai",
    "Juin",
    "Juillet",
    "Août",
    "Septembre",
    "Octobre",
    "Novembre",
    "Décembre",
]
mois_dict = {}
sum_mois_dict = {}
name_col.pop(0)
for i in range(len(mois)):
    date1 = pd.to_datetime("2023-" + str(i + 1) + "-01")
    if i == 11:
        date1 = pd.to_datetime("2022-" + str(i + 1) + "-01")
        date2 = pd.to_datetime("2022-" + str(i + 1) + "-31")
        mois_dict[mois[i]] = df_energies[
            (df_energies["date"] >= date1) & (df_energies["date"] <= date2)
        ]
    else:
        date2 = pd.to_datetime("2023-" + str(i + 2) + "-01")
        mois_dict[mois[i]] = df_energies[
            (df_energies["date"] >= date1) & (df_energies["date"] < date2)
        ]
    mois_dict[mois[i]] = mois_dict[mois[i]].drop(columns=["date"])
    for nrj12 in name_col:
        mois_dict[mois[i]][nrj12] = sum(mois_dict[mois[i]][nrj12])
    mois_dict[mois[i]] = mois_dict[mois[i]].iloc[0]

df_prod_final = mois_dict["Janvier"]
df_prod_final = pd.DataFrame.from_dict(df_prod_final).T
# print(df_prod_final)
for month, df_nrj in mois_dict.items():
    # print(pd.DataFrame.from_dict(df_nrj).T)
    df_prod_final = pd.concat([df_prod_final, pd.DataFrame.from_dict(df_nrj).T])
df_prod_final = df_prod_final.tail(-1)
df_prod_final.insert(0, "Mois", mois, True)
df_prod_final = df_prod_final.set_index("Mois")
# print(df_prod_final)

Affichage

In [ ]:
ax = df_prod_final.plot.bar(stacked=True, figsize=(18, 6))

ax.set_xticklabels(mois, rotation=0)
ax.set_title("Comparaison de la production d'énergie durant l'année")
ax.legend(loc="upper right")

### Conclusion
Grâce à cette représentation on remarque que dans sa globalité la proportion de production d'énergie au cours de l'année reste plutôt constante.
Certaines tendances sont cependant notables, comme la production de charbon qui voit une hausse non négligeable en Décembre ou encore la production de gaz qui se baisse particuliérement en Mai.
Pour finir on peut remarquer que la production d'énergie solaire baisse pendance l'hiver pour remonter en été. Cela s'explique par la durée des jours qui se racourcissent en hiver et se ralong en été.

### 3.5 Quelles sont les périodes de l'année où la production d'énergie est la plus importante/faible ?

### Conclusion
Pour cette question il suffit de reprendre la représentation utilisé par la question précédente. 
On remarque que Janvier est en première position suivi de novembre et décembre. 
Cette hausse de production vient probablement des fêtes de fins d'années.
C'est une période particulièrement rentable pour les entreprises et il est logique de penser que les productions d'énergies suivent cette tendance.
A l'inverse, la période la moins productive se retrouve être le milieu de l'année avec les mois de Juin, Juillet et Août ayant une production équivalente. 
Cette baisse peut s'expliquer par la hausse de chaleur. En été, moins d'énergie est dépensé pour le chauffage et avec cette baisse de consommation la production baisse aussi pour s'adapter à la demande.

### 3.6 Quelles sont les moments de la journée où la production d'énergie est la plus importante/faible ?

Somme des données de production en une seule

In [ ]:
name_col = [
    "date",
    "Charbon",
    "Gaz",
    "Fioul",
    "Nucléaire",
    "Eolien",
    "Solaire",
    "Hydraulique",
    "Bioénergies",
]

df_energies = df_full[name_col]

df_energies = df_energies.dropna()
name_col.pop(0)
df_energies["prod_total"] = df_energies[name_col].sum(axis=1)

date1 = pd.to_datetime("11:02")
time_dict = {}
for ind in df_energies.index:
    heure = df_energies["date"][ind].hour
    if heure not in time_dict.keys():
        time_dict[heure] = df_energies["prod_total"][ind]
    else:
        time_dict[heure] += df_energies["prod_total"][ind]
        # print(f'date: {df_energies["date"][ind]}, conso: {df_energies["conso_total"][ind]}')
print(time_dict)

Affichage

In [ ]:
figure = plt.figure(figsize=(20, 10))
plt.plot(time_dict.keys(), time_dict.values(), label=str(heure))

plt.xlabel("Heure")
plt.ylabel("Production (MW)")
plt.show()

### Conclusion
On remarque que la production d'énergie est fortement influencé par le jour et la nuit. 
La production atteint son pic vers midi pour ensuite redescendre avant d'atteindre son minimum vers 3h du matin.
Il est intéressant de voir que la production suit presque une courbe sinusoidale. Lorsque la production commence à monter, elle continue jusqu'à atteindre sont maximum puis descend jusqu'à atteindre son minimum.

Pour conclure, on ne trouve aucune anomalie dans la production d'énergie et sa courbe est étrangement régulière.

### 3.7 La météo influe-t-elle sur la consomation d'énergie ?


In [ ]:
col_names = [
    "Consommation",
    "température",
    "vitesse_vent",
    "humidité",
    "pression",
    "précipitation_24h",
    "hauteur_neige",
]
# df_full.columns.values.tolist()

df_selection = df_full[col_names]
# removes lines with nan or None
df_selection = df_selection.dropna()

df_selection.describe()

Matrice de corrélation

In [ ]:
corr_matrix = df_selection.corr()

# for only the "Consommation" column sort by descending order of absolute values
pd.DataFrame(
    corr_matrix["Consommation"].sort_values(ascending=False, key=lambda x: abs(x))
)

### Conclusion

On remarque que la température est inversement corrélée avec la consomation d'énergie. Cela est logique car plus il fait froid, plus les gens chauffent leur logement.
Cependant c'est la seule corrélation notable.

### 3.8 Quelles sont les sources de production d'énergie les plus corrélées avec la météo ?

In [ ]:
col_names_prod = [
    "Charbon",
    "Gaz",
    "Fioul",
    "Nucléaire",
    "Eolien",
    "Solaire",
    "Hydraulique",
    "Bioénergies",
]
col_names_weather = [
    "température",
    "vitesse_vent",
    "humidité",
    "pression",
    "précipitation_24h",
    "hauteur_neige",
]

df_selection = df_full[col_names_prod + col_names_weather]
df_selection = df_selection.dropna()

df_selection.describe()

In [ ]:
# create a dataset with with the correlation between the production and the weather for each energy
corr_dataset = pd.DataFrame()
for col_name in col_names_prod:
    current_df = df_selection[[col_name] + col_names_weather]
    corr_matrix = current_df.corr()

    corr_list = corr_matrix[col_name]

    corr_dataset.insert(0, col_name, corr_matrix[col_name])

corr_dataset.dropna()

### Conclusion

On remarque que la température influence toutes les produciont d'énérgie, quand il fait froid la production augmentent et inversement. Sauf pour le solaire qui est inversé car on produit plus d'énergie solaire en été lorsque le soeil est haut.

L'autre corrélation notable est celle entre le solaire et l'humidité. Cela est logique car il y a plus de nuage lorsque l'humidité est élevée et donc moins de soleil. De même pour les temps pluvieux. Cependant la corrélation entre le solaire et les précipitations des dernières 24h est inexistante, ce qui est très étonnant.

L'humidité à également un impact sur la production d'énégie hydrolique. Cela est logique car il y a plus d'eau dans les barages. Cependant elle a également in impact sur l'éolien, le nucléaire, le gaz et le charbon. Cela est plus difficile à expliquer.

La dernière corrélation est entre la vitesse du vent, la pression et la production d'énérgie éolienne qui est tout à fait logique, les pression et dépression entrainne du vent qui favorisent la production d'énergie éolienne.

### 3.9 Classification des jours de l'année par rapport à la consomation d'énergie.
classifié binaire et classifié par la semaine

In [ ]:
date1 = pd.to_datetime("2022-01-01")
date2 = pd.to_datetime("2024-01-01")
df_selection = df_full[(df_full["date"] >= date1) & (df_full["date"] < date2)]

name_col = [
    "date",
    "Consommation",
    "Charbon",
    "Gaz",
    "Fioul",
    "Nucléaire",
    "Eolien",
    "Solaire",
    "Hydraulique",
    "Bioénergies",
]
semaine = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi", "Samedi", "Dimanche"]
df_energies = df_selection[name_col]

df_energies = df_energies.dropna()
day_df = pd.DataFrame()
row_dict = pd.DataFrame()
for ind in df_energies.index:
    jour = (df_energies["date"][ind].day, df_energies["date"][ind].month)
    day_of_week = df_energies["date"][ind].weekday()
    if "jour" not in row_dict.columns:
        row_dict["jour"] = [semaine[day_of_week]]
        for column in name_col:
            row_dict[column] = [df_energies[column][ind]]
    else:
        if row_dict["jour"][0] != semaine[day_of_week]:
            day_df = pd.concat([day_df, row_dict], ignore_index=True)
            row_dict["jour"] = semaine[day_of_week]
            for column in name_col:
                row_dict[column] = df_energies[column][ind]
        else:
            for column in name_col:
                if column != "date":
                    row_dict[column] += df_energies[column][ind]
                    row_dict[column] /= 2

print(day_df)

Affichage de la consommation par jour

In [ ]:
figure = plt.figure(figsize=(20, 10))

plt.plot(day_df["date"], day_df["Consommation"])
plt.xlabel("Date")
plt.ylabel("Consommation")
plt.show()

### Conclusion
La consommation suit les même variation que la production d'énergie.
Elle est haute en hiver et durant les fêtes mais reste basse en été.
On retrouve donc cette correlation avec non seulement les saisons mais aussi la logique des entreprises qui boost leur ventes en fin d'années et donc leur production.

Affichage de la consommation par jour de la semaine

In [ ]:
df_week_days = pd.DataFrame()
interesting_col = ["jour", "Consommation"]
day_df_filtered = day_df[interesting_col]

for ind in day_df_filtered.index:
    jour = day_df_filtered["jour"][ind]
    
    if df_week_days.empty:
        df_week_days["jour"] = [day_df_filtered["jour"][ind]]
        df_week_days["Consommation"] = [day_df_filtered["Consommation"][ind]]
    # print(df_week_days)
    else:
        if jour not in df_week_days["jour"].values:
            df_week_days.loc[len(df_week_days)] = day_df_filtered.iloc[ind]
        else:
            day_index = df_week_days.index[df_week_days["jour"] == jour][0]
            df_week_days["Consommation"][day_index] += day_df_filtered["Consommation"][
                ind
            ]
            df_week_days["Consommation"][day_index] /= 2

df_week_days.index = [
    6,
    0,
    1,
    2,
    3,
    4,
    5,
]  # faire en sorte que les jours soient dans l'ordre
df_week_days = df_week_days.sort_index()
print(df_week_days)

figure = plt.figure(figsize=(10, 5))

plt.ylim([50000, 60000])
plt.bar(df_week_days["jour"], df_week_days["Consommation"])
plt.xlabel("Jour de la semaine")
plt.ylabel("Consommation (MW)")
plt.show()

### Conclusion
La fin de la semaine représente une baisse notable dans la consommation car les salariés sont en weekend et les entreprises baisses leur production. 
Il est quand même intéressant de voir que cette baisse se poursuit jusqu'au lundi qui a une consommation certe plus haute qu'en weekend mais assez basse comparé au milieu de semaine.
Globalement le milieu de semaine reste la période consommation la plus élevé et les autres jours sont influencés par la baisse notable du weekend.

# 4. Prédiction

### 4.1 Regrouper les données par période de temps jours/mois/années

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

date1 = pd.to_datetime("2022-06-01")
date2 = pd.to_datetime("2023-12-01")

col_names = [
    "Consommation",
    "Production",
    "Bioénergies",
    "Hydraulique",
    "Solaire",
    "Eolien",
    "Nucléaire",
    "Fioul",
    "Gaz",
    "Charbon",
]

# df_selection = df_full[(df_full['date'] >= date1) & (df_full['date'] < date2)]
df_selection = df[(df["date"] >= date1) & (df["date"] < date2)]
df_selection = df_selection.dropna()

X = df_selection[col_names]
y = df_selection["Nucléaire"]

# print(X)

kmeans = KMeans(n_clusters=2, random_state=0).fit(X)

kmeans.labels_

y_pred = KMeans(n_clusters=4).fit_predict(X)

figure = plt.figure(figsize=(20, 10))
plt.scatter(df_selection["Date"], X["Consommation"], c=y_pred)
plt.title("Clustering")
plt.xlabel("Date")
plt.ylabel("(MW)")
plt.show()

# kmeans.predict([[0, 0], [12, 3]])
# kmeans.cluster_centers_

In [ ]:
heures = []
cluster1 = []
cluster2 = []
cluster3 = []
cluster4 = []
df_selection["y_pred"] = y_pred
for i in range(24):
    heures.append(str(i))
    df_temp = df_selection[pd.to_datetime(df_selection["Heures"]).dt.hour == i]

    cluster1.append(len(df_temp[df_selection["y_pred"] == 0]))
    cluster2.append(len(df_temp[df_selection["y_pred"] == 1]))
    cluster3.append(len(df_temp[df_selection["y_pred"] == 2]))
    cluster4.append(len(df_temp[df_selection["y_pred"] == 3]))

dict = {
    "Heure": heures,
    "cluster1": cluster1,
    "cluster2": cluster2,
    "cluster3": cluster3,
    "cluster4": cluster4,
}
df_cluster = pd.DataFrame(dict)

figure = plt.figure(figsize=(20, 10))
df_cluster.plot.bar(stacked=True)

for i in range(4):
    print(
        "moyenne cluster",
        i,
        df_selection[df_selection["y_pred"] == i]["Consommation"].mean(),
    )
# d.plot.bar(stacked=True, x="Heures")

## 4.2 Peut-on classifier les jours de la semaine par rapport à la consomation et la production d'énergie ?

On cré un modèle qui a partir des données de production d'énergie et de consommation d'énergie, nous dit si c'est un jour de la semaine ou du weekend.

On commence par ajouter une collone jour de la semaine dans le dataset et on supprime la date.

In [ ]:
df_selection = df_full

name_col = [
    "date",
    "Consommation",
    "Charbon",
    "Gaz",
    "Fioul",
    "Nucléaire",
    "Eolien",
    "Solaire",
    "Hydraulique",
    "Bioénergies",
]
semaine = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi", "Samedi", "Dimanche"]
df_energies = df_selection[name_col]

df_energies = df_energies.dropna()
day_df = pd.DataFrame()
row_dict = pd.DataFrame()
for ind in df_energies.index:
    jour = (df_energies["date"][ind].day, df_energies["date"][ind].month)
    day_of_week = df_energies["date"][ind].weekday()
    if "jour" not in row_dict.columns:
        row_dict["jour"] = [semaine[day_of_week]]
        for column in name_col:
            row_dict[column] = [df_energies[column][ind]]
    else:
        if row_dict["jour"][0] != semaine[day_of_week]:
            day_df = pd.concat([day_df, row_dict], ignore_index=True)
            row_dict["jour"] = semaine[day_of_week]
            for column in name_col:
                row_dict[column] = df_energies[column][ind]
        else:
            for column in name_col:
                if column != "date":
                    row_dict[column] += df_energies[column][ind]
                    row_dict[column] /= 2

data = day_df

data = data.drop(["date"], axis=1)
data.head()

On transforme les données catégorielles en données numériques (binaire, 0: jour de la semaine, 1: weekend).

In [ ]:
""""
day_dict = {
    "Lundi": 0,
    "Mardi": 1,
    "Mercredi": 2,
    "Jeudi": 3,
    "Vendredi": 4,
    "Samedi": 5,
    "Dimanche": 6,
}
"""

day_dict = {
    "Lundi": 0,
    "Mardi": 0,
    "Mercredi": 0,
    "Jeudi": 0,
    "Vendredi": 0,
    "Samedi": 1,
    "Dimanche": 1,
}

data["jour"] = data["jour"].map(day_dict)
data.head()

Séparation des données en données d'entrainement et de test.

In [ ]:
from sklearn.model_selection import train_test_split

X, Y = data.iloc[:, 1:], data["jour"]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
print(len(x_train), len(y_train), len(x_test), len(y_test))

Choix du modèle

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn import linear_model

model = DecisionTreeClassifier(random_state=0)
# model = SVC(gamma='auto')
# model = linear_model.Lasso(alpha=0.1)

Entainement du modèle

In [ ]:
model = model.fit(x_train, y_train)

Matrice de confusion

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = model.predict(x_test)

cf_matrix = confusion_matrix(y_test, y_pred)
print(cf_matrix)

Validation croisée

In [ ]:
import matplotlib.pyplot as plt

score = model.predict_proba(x_test)
type(score)

plt.hist(score[:, 0], bins=25, alpha=0.5, color="red")
plt.hist(score[:, 1], bins=25, alpha=0.5, color="blue")

plt.title("histogram of the score")
plt.legend(["False", "True"])

plt.show()

Courbe ROC

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(y_test, score[:, 1], pos_label=1)
print(fpr), print(tpr)
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()
plt.show()

AUC

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, score[:, 1], multi_class="ovr")

Validation croissée

In [ ]:
from numpy import mean, std
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, shuffle=True)

best_area = 0
best_model = None

areas = []
for i, (train_index, test_index) in enumerate(kf.split(X)):
    model = DecisionTreeClassifier(max_depth=5)

    model = model.fit(X.iloc[train_index], Y[train_index])

    y_test_kf = Y[test_index]
    score = model.predict_proba(X.iloc[test_index])

    area = roc_auc_score(y_test_kf, score[:, 1])

    areas.append(area)

    if area > best_area:
        best_area = area
        best_model = model

print(areas)
areas = np.array(areas)
print("mean :", mean(areas), "; std :", std(areas))

print("Best area :", best_area)

### Conclusion

On obtient un score de 0.77 ce qui est plutôt bon. On peut donc conclure que notre modèle est satisfaisant.
On peut donc prédire si un jour est un jour de la semaine ou du weekend à partir de la production et de la consommation d'énergie. Cela parait logique car on a pu observé précédement que la consomation et la produciton était plus faible en weekend (bien que la production varie toute l'année).